# Source localization - Stage1: MRI segmentation
- A working installation of freesurfer is needed



#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

# Subject selection + Configuration

In [ ]:
from src import SubjectDropDowner
import os
import glob

input_dir = "/home/meg/Schreibtisch/new_patients"  # Top level folder containing MRI files
processorcores = 15    # How many CPU cores to use for the computation 
                        # MEG-Workstation -> use 15
                        # at home -> use 1

def get_subject_list(input_dir = input_dir) -> list:
    analist = glob.glob(os.path.join(input_dir, "*"))
    analist = [os.path.basename(f) for f in analist if os.path.isdir(f)]
    analist = sorted([f for f in analist])
    return analist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nThe following patients/ subjects are available:")
drop_menu

# Freesurfer

In [ ]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    ject = subject
    subject = "sub-" + subject
else:
    subject = subject
    ject = subject.replace("sub-", "")

print(f"Ject: {ject}, Subject: {subject}")

def get_dicom_file(subject) -> str:
    if not os.path.isdir(os.path.join(input_dir, ject)):
        print(f"No anatomical data found for {subject}, aborting\n\n")
        raise(FileNotFoundError)
    else:
        dicom_path = os.path.join(input_dir, subject, "1*", "100*", "1*", "*")
        dicom = str(glob.glob(dicom_path, recursive=True)[0])
        dicom = os.path.abspath(dicom)
        return dicom

def get_dicom_path(subject) -> str:
    dicom = get_dicom_file(subject)
    return dicom

def get_watershed_comand(subject) -> str:
    return f'python -c "import mne; mne.bem.make_watershed_bem(subject=\'{subject}\')"'

def get_recon_all_command(ject) -> str:
    dicom = get_dicom_path(ject)
    subject = "sub-" + ject
    command = f"recon-all -s {subject} -i {dicom} -all -openmp {processorcores} && segmentHA_T1.sh {subject} && {get_watershed_comand(subject)}" 
    return command

command = get_recon_all_command(ject)

print(f"\n\nExecute the following command in the bash shell:\n \
      (This will take hours...)\n\n{command}\n\n")